# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

Импортируем нужные библиотеки для работы 

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Можно изменить в столбце 'messages' тип данных на int, так как кол-во сообщений не дробные.

In [4]:
df['messages'] = df['messages'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 125.7 KB


**Вывод:**
- В файле 3214 строк и 5 столбов
- Изменили тип данных столбца 'messages' с 'float' на 'int' 

## Разбейте данные на выборки

Разбиваем данные на признаки и целевые признаки

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Разбиваем данные на 3 выборки: тренировочную, тестовую и валидационнную

In [6]:
features_train, tes_val_features, target_train, tes_val_target = train_test_split(features, 
                                                                              target, 
                                                                              test_size = 0.4, 
                                                                              random_state = 123) 

features_test, features_valid, target_test, target_valid = train_test_split(tes_val_features,
                                                                            tes_val_target,
                                                                            test_size = 0.5,
                                                                            random_state = 123)                                                        

In [7]:
print('Тренировочная выборка:', features_train.shape)
print('Валидационная выборка:', features_valid.shape)
print('Тестовая выборка:', features_test.shape)

Тренировочная выборка: (1928, 4)
Валидационная выборка: (643, 4)
Тестовая выборка: (643, 4)


**Вывод:**
- Разбили датасет на признаки и целевые признаки для моделирования 
- Разделили исходный набор данных на тренировочную, валидационную и тестовую выборки в соотношении 60%-20%-20%.

## Исследуйте модели

Для исследования выберем такие модели как:
- Дерево решений
- Случайный лес
- Логистическая регрессия   

Дерево решений

Проверим для дерева решений несколько гиперпараметров: 'max_depth', 'min_samples_split' и 'min_samples_leaf' c помощью с функции GridSearchCV

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth' : range(1,10),
    'min_samples_split' : range(2,12),
    'min_samples_leaf' : range(1,6)
}

grid_search = GridSearchCV(estimator = DecisionTreeClassifier(random_state = 123), 
                           param_grid = param_grid, 
                           scoring = 'accuracy', 
                           cv = 5)

grid_search.fit(features_train, target_train)
print('Best accuracy:', grid_search.best_score_)
print('Best parameters:', grid_search.best_params_)

Best accuracy: 0.7909723437184576
Best parameters: {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 9}


Проверим лучшие гиперпараметры на валидационной выборке

In [10]:
best_model = grid_search.best_estimator_
best_model.fit(features_train, target_train)
predictions_valid = best_model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions_valid)
print('Accuracy наилучшей модели на валидационной выборке:', accuracy)

Accuracy наилучшей модели на валидационной выборке: 0.7791601866251944


Проверил также через циклы и получил другие гиперпараметры и результат

In [ ]:
best_model = None
best_accuracy = 0

for max_depth in tqdm (range(1, 10)):
    for min_samples_split in range(2, 12):
        for min_samples_leaf in range(1, 6):
            model = DecisionTreeClassifier(random_state=123,
                                           max_depth=max_depth,
                                           min_samples_split=min_samples_split,
                                           min_samples_leaf=min_samples_leaf)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            accuracy = accuracy_score(target_valid, predictions)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
                best_params = {'max_depth': max_depth,
                               'min_samples_split': min_samples_split,
                               'min_samples_leaf': min_samples_leaf}

print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy) 
print("Лучшие параметры:", best_params)

Случайный лес

In [ ]:
from tqdm import tqdm
best_model = None
best_result = 0

for est in tqdm(range (10, 100, 10)):
    for depth in range (1, 10):
        for min_samples_split in range(2, 12):
            for min_samples_leaf in range(1, 6):
                model = RandomForestClassifier(random_state = 123, 
                                               n_estimators = est, 
                                               max_depth = depth, 
                                               min_samples_split = min_samples_split,
                                               min_samples_leaf = min_samples_leaf)
                model.fit(features_train, target_train)
                predictions_valid = model.predict(features_valid)
                result = accuracy_score(target_valid, predictions_valid) 
                if result > best_result:
                    best_model = model
                    best_result = result
                    best_depth = depth
                    best_est = est
                    best_min_samples_split = min_samples_split
                    best_min_samples_leaf = min_samples_leaf
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result) 
print("Число деревьев (est) наилучшей модели на валидационной выборке:", best_est)
print("Глубина дерева наилучшей модели на валидационной выборке:", best_depth) 
print("Число узлов наилучшей модели на валидационной выборке:", best_min_samples_split) 
print("Число листьев наилучшей модели на валидационной выборке:", best_min_samples_leaf) 

Логистическая регрессия 

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

model = LogisticRegression(random_state=123, solver='lbfgs', max_iter=100)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(features_train, target_train)

print('Best accuracy:', grid_search.best_score_)
print('Best parameters:', grid_search.best_params_) 

С помощью функции GridSearchCV и переборов параметров 'C' и 'penalty' выявили лучшую модель Логистической регрессии:
- Best accuracy: 0.7443106116681246
- Best parameters: {'C': 100, 'penalty': 'l2'}

**Вывод:**
- Обучили модель древа решений и проверили ее на валидационной выборке с целью выявить наилучшие параметры.
- Обучили модель случайного леса и проверили ее на валидационной выборке с целью выявить наилучшие параметры.
- Обучили модель логистической регрессии и проверили ее на валидационной выборке.

## Проверьте модель на тестовой выборке

Дерево решений

In [ ]:
model = DecisionTreeClassifier(random_state = 123, 
                               max_depth = 3,
                               min_samples_split = 2, 
                               min_samples_leaf = 1)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print('Точность тестовой модели дерева решений:', accuracy)

Случайный лес

In [ ]:
model = RandomForestClassifier(random_state = 123,
                               n_estimators = 10, 
                               max_depth = 6,
                               min_samples_split = 2, 
                               min_samples_leaf = 1)
model.fit(features_train, target_train)
accuracy_best_model = model.score(features_test, target_test)
print('Точность тестовой модели случайного леса:', accuracy)

Логистическая регрессия

In [ ]:
model = LogisticRegression(random_state = 123, solver='lbfgs', max_iter=100)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print('Accuracy на валидационной выборке:', result)   

**Вывод:**
- Проверили модель дерева решений с наилучшими параметрами на тестовой выборке
- Проверили модель случайного леса с наилучшими параметрами на тестовой выборке
- Проверили модель логистической регрессии на тестовой выборке
- Наилучшая модель с самым высоким показателем accuracy - **Случайный лес** (accuracy = 0.80248833592535) с геперпараметрами: n_estimators = 10, max_depth = 6, Число узлов наилучшей модели на валидационной выборке: 2, min_samples_split: 2, min_samples_leaf: 1.

## (бонус) Проверьте модели на адекватность

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='stratified', random_state=123)
dummy_model.fit(features_train, target_train)
dummy_model_accuracy = dummy_model.score(features_valid, target_valid)

print('Accuracy простейшей модели:', dummy_model_accuracy)
print('Accuracy наилучшей модели на валидационной выборке:', accuracy_best_model)

Для проверки на адекватность используем метод из библиотеки sklearn - DummyClassidier, используем параметр - stratified, который предсказывает классы с вероятностями, пропорциональными частоте встречаемости классов в обучающей выборке. 

**Вывод:**
Наша модель на порядок точнее простейшей модели, а это значит, что наша модель прошла тест на адекватность.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

# Вывод
**Знакомство с данными:**
- В файле 3214 строк и 5 столбов
- Изменили тип данных столбца 'messages' с 'float' на 'int' 

**Подготовка данных для моделирования:**
- Разбили датасет на признаки и целевые признаки для моделирования 
- Разделили исходный набор данных на тренировочную, валидационную и тестовую выборки в соотношении 60%-20%-20%.

**Исследование моделей:**
- Обучили модель древа решений и проверили ее на валидационной выборке с целью выявить наилучшие параметры.
- Обучили модель случайного леса и проверили ее на валидационной выборке с целью выявить наилучшие параметры.
- Обучили модель логистической регрессии и проверили ее на валидационной выборке.

**Тестирование моделей:**
- Проверили модель дерева решений с наилучшими параметрами на тестовой выборке
- Проверили модель случайного леса с наилучшими параметрами на тестовой выборке
- Проверили модель логистической регрессии на тестовой выборке
- Наилучшая модель с самым высоким показателем accuracy - **Случайный лес** (accuracy = 0.80248833592535) с геперпараметрами: n_estimators = 10, max_depth = 6, Число узлов наилучшей модели на валидационной выборке: 2, min_samples_split: 2, min_samples_leaf: 1.